In [4]:
import pandas as pd
import time
import numpy as np

# Static Data

In [5]:
agency = pd.read_csv(r"../rome_static_gtfs/agency.txt", sep = ",")

In [6]:
routes = pd.read_csv(r"../rome_static_gtfs/routes.txt", sep = ",")
atac_routes = routes.loc[routes["agency_id"] == "OP1"][["route_id","route_short_name","route_long_name","route_type"]]
atac_routes = atac_routes["route_id"].drop_duplicates()
atac_routes.shape
routes

,route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color
0,211,OP1,211,NaN,3,NaN,NaN,NaN
1,C2,OP1,C2,NaN,3,NaN,NaN,NaN
2,62,OP1,62,NaN,3,NaN,NaN,NaN
3,105,OP1,105,NaN,3,NaN,NaN,NaN
4,766,OP1,766,NaN,3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
426,4357,OP2,787,NaN,3,http://muovi.roma.it/percorso/js?query=787&cl=1,NaN,NaN
427,312,OP2,665,NaN,3,http://muovi.roma.it/percorso/js?query=665&cl=1,NaN,NaN
428,345,OP2,437,NaN,3,http://muovi.roma.it/percorso/js?query=437&cl=1,NaN,NaN
429,3632,OP2,235,NaN,3,http://muovi.roma.it/percorso/js?query=235&cl=1,NaN,NaN


In [5]:
len(routes), len(atac_routes)

(431, 328)

In [6]:
dtypes = {
    'route_id': str,
    'service_id': str,
    'trip_id': str,
    'trip_headsign': str,
    'trip_short_name': str,
    'direction_id': str,
    'block_id': str,
    'shape_id': str,
    'wheelchair_accessible': str,
    'exceptional': str
}
trips = pd.read_csv(r"../rome_static_gtfs/trips.txt", sep = ",")[["route_id","service_id","trip_id","trip_headsign","block_id"]]
atac_trips = trips.merge(atac_routes, on = "route_id", how = "inner")
atac_trips

,route_id,service_id,trip_id,trip_headsign,block_id
0,T01,1#28,1#1-2,TUSCOLANA/ANAGNINA,NaN
1,T01,1#51,1#2-2,TUSCOLANA/ANAGNINA,NaN
2,T01,1#29,1#3-2,TUSCOLANA/ANAGNINA,NaN
3,T01,1#29,1#4-2,TUSCOLANA/ANAGNINA,NaN
4,T01,1#28,1#5-2,TUSCOLANA/ANAGNINA,NaN
...,...,...,...,...,...
148428,342,150,VJbfab8bd45428b9173429174015c36f29a6bbc6e6,Pantano,NaN
148429,342,150,VJ3d7ef6c7ea47a7970b34d5f46b042b677e66334d,Pantano,NaN
148430,342,150,VJb432db5eff2ad42fa1de4682eb37a25d63ffadc2,Pantano,NaN
148431,342,150,VJ24a34d64e904499a443bd541e326a369454489ab,Pantano,NaN


In [7]:
len(trips),len(atac_trips)

(172480, 148433)

In [11]:
calendar_dates = pd.read_csv(r"../rome_static_gtfs/calendar_dates.txt", sep = ",")
#USED actualy
calendar_dates = calendar_dates.loc[calendar_dates["exception_type"] ==1]["service_id"].drop_duplicates()

In [24]:
active_atac_trips = atac_trips.merge(calendar_dates, on = "service_id", how = "inner")

In [25]:
active_atac_trips = active_atac_trips[['route_id', 'service_id', 'trip_id', 'trip_headsign']].drop_duplicates()

In [30]:
len(active_atac_trips),active_atac_trips["trip_id"].nunique()

(145918, 145918)

In [31]:
stops = pd.read_csv(r"../rome_static_gtfs/stops.txt", sep = ",")
stops = stops["stop_id"].drop_duplicates()
stops.head(5)

0    00213
1    04972
2    04994
3    05000
4    05001
Name: stop_id, dtype: object

In [32]:
# Specify data types for columns 3 and 5 as str (string)
dtypes = {'trip_id': str,'stop_id': str, 'stop_headsign': str}
stop_times = pd.read_csv(r"../rome_static_gtfs/stop_times.txt", sep = ",", dtype=dtypes)
# Convert the 'arrival_time' and 'departure_time' columns to datetime with the specified format
stop_times['arrival_time'] = pd.to_datetime(stop_times['arrival_time'], format='%H:%M:%S', errors='coerce').dt.time
stop_times['departure_time'] = pd.to_datetime(stop_times['departure_time'], format='%H:%M:%S', errors='coerce').dt.time

# Filter out rows with invalid times (e.g., "25:00:00" becomes NaT)
stop_times = stop_times.dropna(subset=['arrival_time', 'departure_time'])
stop_times = stop_times.merge(stops, on = "stop_id", how = "inner")
stop_times.head(5)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,1#1-2,07:30:00,07:30:00,74761,1,NaN,NaN,NaN,0,1
1,1#2-2,07:28:00,07:28:00,74761,1,NaN,NaN,NaN,0,1
2,1#3-2,07:30:00,07:30:00,74761,1,NaN,NaN,NaN,0,1
3,1#4-2,12:33:00,12:33:00,74761,1,NaN,NaN,NaN,0,1
4,1#5-2,12:32:00,12:32:00,74761,1,NaN,NaN,NaN,0,1


In [33]:
len(stop_times)

4907776

In [34]:
atac_state = stop_times.merge(active_atac_trips, on = "trip_id", how = "inner")
atac_state.shape

(4049478, 13)

In [35]:
# Define the time range
from datetime import datetime, time
start_time_range = datetime.strptime('14:00:00', "%H:%M:%S").time()
end_time_range = datetime.strptime('15:40:00', "%H:%M:%S").time()

# Apply the filter
atac_state_100 = atac_state[(atac_state['arrival_time'] >= start_time_range) & (atac_state['arrival_time'] <= end_time_range) &
                 (atac_state['departure_time'] >= start_time_range) & (atac_state['departure_time'] <= end_time_range)]

atac_state_100 = atac_state_100[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'shape_dist_traveled',
       'timepoint', 'route_id', 'service_id', 'trip_headsign']]
atac_state_100.to_csv("../output_data/atac_state_100.csv", index = False)

In [36]:
atac_state_100.head(3)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,shape_dist_traveled,timepoint,route_id,service_id,trip_headsign
130,1#2138-15,15:02:00,15:02:00,74761,1,NaN,0,1,T01,1#5,ANAGNINA (MA)
131,1#2138-15,15:05:46,15:05:46,74763,2,NaN,1167,0,T01,1#5,ANAGNINA (MA)
132,1#2138-15,15:07:28,15:07:28,74765,3,NaN,1697,0,T01,1#5,ANAGNINA (MA)


In [39]:
atac_state_full = atac_state[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'shape_dist_traveled',
       'timepoint', 'route_id', 'service_id', 'trip_headsign']]
atac_state_full = atac_state_full.loc[~atac_state_full["trip_id"].str.startswith("VJ")]
atac_state_full.to_csv("../output_data/atac_state_full.csv", index = False)

In [40]:
atac_state_full.shape

(4039455, 11)

# Measuring Performance
- **Measure average delay**
- 